In [7]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
from sklearn.model_selection import train_test_split
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

##
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
from sklearn.metrics import classification_report

In [137]:
test_in = pd.read_csv('/content/test_input_example.csv')
test_out = pd.read_csv('/content/test_output_example.csv')
full_test1 = [test_in,test_out]
full_test = pd.concat(full_test1,axis = 1)
full_test = full_test.drop(['admission_datetime','PATIENT ID'], axis=1)
final_data = pd.read_csv("/content/fixed_test.csv")

In [138]:
final_id = final_data['PATIENT ID']

In [139]:
data = pd.read_csv('/content/hm_hospitales_covid_structured_30d_train.csv')
y = pd.read_csv('/content/split_train_export_30d.csv')
data_1 = [data,y]
data_val = pd.concat(data_1,axis = 1)

In [140]:
title_mapping1 = {"sx_breathing_difficulty": 1, "sx_cough": 2, "sx_fever": 3, "sx_flu": 4, "sx_others": 5}
title_mapping2 = {"MALE": 1,"FEMALE":0}
data_val['ed_diagnosis'] = data_val['ed_diagnosis'].map(title_mapping1)
full_test['ed_diagnosis'] = full_test['ed_diagnosis'].map(title_mapping1)
final_data['ed_diagnosis'] = final_data['ed_diagnosis'].map(title_mapping1)
data_val['sex'] = data_val['sex'].map(title_mapping2)
full_test['sex'] = full_test['sex'].map(title_mapping2)
final_data['sex'] = final_data['sex'].map(title_mapping2)
data_val = data_val.drop(['admission_datetime','PATIENT ID'], axis=1)
final_data = final_data.drop(['admission_datetime','PATIENT ID'], axis=1)


In [141]:
columns = list(data_val.columns)
for (i,col) in zip(range(len(columns)),columns):
  if data_val.iloc[:,i].isnull().values.any()== True:
    data_val.iloc[:,i].fillna(data_val[col].median(), inplace = True)
data_val.head()

,age,sex,ed_diagnosis,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia,lab_aptt,lab_ldh,lab_sodium,lab_alt,lab_ddimer,lab_glucose,lab_potassium,lab_mch,lab_mean_platelet_volume,lab_inr,lab_hct,lab_creatinine,lab_ast,lab_crp,lab_mcv,lab_leukocyte,lab_lymphocyte_percentage,lab_prothrombin_activity,lab_urea,lab_rbc,lab_platelet,lab_neutrophil,lab_neutrophil_percentage,lab_lymphocyte,lab_rdw,lab_hemoglobin,hospital_outcome
0,72,1,1,37.2,111.0,63.0,94.0,60.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.3,593.00,135.0,118.00,908.0,124.0,4.34,32.9,11.2,1.21,36.0,0.94,45.0,95.04,96.3,11.600,5.0,70.0,67.0,3.74,238.0,10.03,86.5,0.62,13.0,13.8,0
1,50,0,1,37.2,131.0,76.0,93.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.4,430.60,138.2,27.40,234.0,91.1,4.57,27.1,10.0,1.15,39.8,0.67,25.8,23.93,82.4,5.000,28.0,78.0,26.4,4.83,197.0,3.41,68.2,1.40,13.9,13.1,0
2,78,0,1,36.4,146.0,96.0,85.0,96.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,522.75,137.1,25.65,744.0,113.6,4.20,29.8,10.3,1.19,41.1,0.91,31.6,68.36,88.2,6.635,16.9,76.0,34.8,4.67,203.0,4.83,74.0,1.05,12.9,13.9,0
3,50,1,1,37.7,131.0,76.0,124.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30.8,694.40,136.5,66.60,416.0,96.2,4.39,27.6,10.0,1.24,44.8,0.88,83.1,71.26,84.2,6.160,20.2,69.0,40.9,5.32,178.0,4.60,74.6,1.18,13.3,14.7,0
4,38,1,1,36.6,131.0,76.0,88.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.1,790.10,135.1,62.70,284.0,105.2,3.69,30.0,9.7,1.11,39.2,0.77,79.5,128.42,87.9,7.020,16.5,89.0,12.6,4.46,298.0,4.92,70.1,1.16,10.6,13.4,0


In [142]:
X1_sample = full_test.drop("hospital_outcome", axis=1)
y1_sample = full_test['hospital_outcome']
X = data_val.drop("hospital_outcome", axis=1)
y = data_val['hospital_outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [143]:
new_col = list(data_val.columns)
fs = SelectKBest(score_func=f_classif, k='all')
fs.fit(X, y)
#X_train_fs = fs.transform(X_train)
#X_test_fs = fs.transform(X_test)
drop_list = []
for i in range(len(fs.scores_)):
  print(new_col[i],'Feature %d: %f' % (i, fs.scores_[i]))
  if (fs.scores_[i] < 5 ) :
    #print(new_col[i],' needs to be dropped!!!!')
    drop_list.append(new_col[i])
data_val = data_val.drop(drop_list, axis=1)
full_test = full_test.drop(drop_list, axis=1)
final_data = final_data.drop(drop_list, axis=1)

age Feature 0: 244.188164
sex Feature 1: 8.003169
ed_diagnosis Feature 2: 0.002782
vitals_temp_ed_first Feature 3: 0.009485
vitals_sbp_ed_first Feature 4: 1.082865
vitals_dbp_ed_first Feature 5: 4.700089
vitals_hr_ed_first Feature 6: 0.199576
vitals_spo2_ed_first Feature 7: 167.396801
pmhx_diabetes Feature 8: 11.223117
pmhx_hld Feature 9: 6.254769
pmhx_htn Feature 10: 9.293561
pmhx_ihd Feature 11: 15.883852
pmhx_ckd Feature 12: 21.898125
pmhx_copd Feature 13: 14.063111
pmhx_asthma Feature 14: 0.387873
pmhx_activecancer Feature 15: 13.189400
pmhx_chronicliver Feature 16: 5.524476
pmhx_stroke Feature 17: 21.677858
pmhx_chf Feature 18: 10.024505
pmhx_dementia Feature 19: 40.107067
lab_aptt Feature 20: 0.420109
lab_ldh Feature 21: 97.081321
lab_sodium Feature 22: 42.439504
lab_alt Feature 23: 0.816220
lab_ddimer Feature 24: 32.409382
lab_glucose Feature 25: 50.594079
lab_potassium Feature 26: 9.528075
lab_mch Feature 27: 7.759471
lab_mean_platelet_volume Feature 28: 13.224878
lab_inr Featu

In [144]:
correlated = pd.DataFrame(columns=['features','correlated_scores'])
correlated.features = new_col[:46]
correlated.correlated_scores = fs.scores_
correlated = correlated.transpose()
correlated

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
features,age,sex,ed_diagnosis,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia,lab_aptt,lab_ldh,lab_sodium,lab_alt,lab_ddimer,lab_glucose,lab_potassium,lab_mch,lab_mean_platelet_volume,lab_inr,lab_hct,lab_creatinine,lab_ast,lab_crp,lab_mcv,lab_leukocyte,lab_lymphocyte_percentage,lab_prothrombin_activity,lab_urea,lab_rbc,lab_platelet,lab_neutrophil,lab_neutrophil_percentage,lab_lymphocyte,lab_rdw,lab_hemoglobin
correlated_scores,244.188,8.00317,0.00278171,0.00948491,1.08286,4.70009,0.199576,167.397,11.2231,6.25477,9.29356,15.8839,21.8981,14.0631,0.387873,13.1894,5.52448,21.6779,10.0245,40.1071,0.420109,97.0813,42.4395,0.81622,32.4094,50.5941,9.52808,7.75947,13.2249,21.7873,1.39344,81.6976,12.5332,98.1986,57.6959,75.0653,109.55,21.7866,209.876,16.0065,6.39546,111.185,109.832,4.94311,43.4192,9.91406


In [145]:
drop_list

['ed_diagnosis',
 'vitals_temp_ed_first',
 'vitals_sbp_ed_first',
 'vitals_dbp_ed_first',
 'vitals_hr_ed_first',
 'pmhx_asthma',
 'lab_aptt',
 'lab_alt',
 'lab_hct',
 'lab_lymphocyte']

In [146]:
X1_sample = full_test.drop("hospital_outcome", axis=1)
y1_sample = full_test['hospital_outcome']
X = data_val.drop("hospital_outcome", axis=1)
y = data_val['hospital_outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify = y)
full_train1 = [X_train,y_train]
full_train = pd.concat(full_train1,axis = 1)
#X_train = X[:1400]
#X_test = X[1400:1800]
#y_train = y[:1400]
#y_test = y[1400:1800]

In [147]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
clf.score(X_train,y_train)
print(classification_report(y_test, Y_pred))

[[462  12]
 [ 50  27]]
0.6923076923076923
0.35064935064935066
              precision    recall  f1-score   support

           0       0.90      0.97      0.94       474
           1       0.69      0.35      0.47        77

    accuracy                           0.89       551
   macro avg       0.80      0.66      0.70       551
weighted avg       0.87      0.89      0.87       551



In [149]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state = 42)
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
clf.score(X_train,y_train)
print(classification_report(y_test, Y_pred))

[[464  10]
 [ 52  25]]
0.7142857142857143
0.3246753246753247
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       474
           1       0.71      0.32      0.45        77

    accuracy                           0.89       551
   macro avg       0.81      0.65      0.69       551
weighted avg       0.87      0.89      0.87       551



In [176]:
from sklearn.utils import resample


# separate minority and majority classes
#data_val1 = data_val[:1400]
not_survived = full_train[full_train.hospital_outcome==0]
survived = full_train[full_train.hospital_outcome==1]

# upsample minority
survive_upsampled = resample(survived,
                          replace=True, # sample with replacement
                          n_samples=len(not_survived), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_survived, survive_upsampled])

# check new class counts
upsampled.hospital_outcome.value_counts()


1    1103
0    1103
Name: hospital_outcome, dtype: int64

In [177]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
X_oversample, y_oversample = smote.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [158]:
from sklearn.ensemble import RandomForestClassifier

clf4 = RandomForestClassifier(random_state = 42)
clf4.fit(X_oversample, y_oversample)
Y_pred = clf4.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
clf.score(X_train,y_train)
print(classification_report(y_test, Y_pred))

[[446  28]
 [ 40  37]]
0.5692307692307692
0.4805194805194805
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       474
           1       0.57      0.48      0.52        77

    accuracy                           0.88       551
   macro avg       0.74      0.71      0.73       551
weighted avg       0.87      0.88      0.87       551



In [154]:
y1_train = upsampled.hospital_outcome
X1_train = upsampled.drop('hospital_outcome', axis=1)

from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=0)
clf.fit(X1_train, y1_train)
Y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
clf.score(X_train,y_train)
print(classification_report(y_test, Y_pred))

[[398  76]
 [ 24  53]]
0.4108527131782946
0.6883116883116883
              precision    recall  f1-score   support

           0       0.94      0.84      0.89       474
           1       0.41      0.69      0.51        77

    accuracy                           0.82       551
   macro avg       0.68      0.76      0.70       551
weighted avg       0.87      0.82      0.84       551



In [155]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state = 42)
clf.fit(X1_train, y1_train)
Y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
clf.score(X_train,y_train)

[[451  23]
 [ 51  26]]
0.5306122448979592
0.33766233766233766


0.9976617303195635

##This is for hyperparameter tuning, cv requires about 20 mins, I have included the hyperparameters already


In [93]:
'''
from sklearn.model_selection import RandomizedSearchCV
clf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

pprint(random_grid)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores

clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
clf_random.fit(X1_train, y1_train)
print(clf_random.best_params_)
'''

{'max_depth': [100, 140, 180, 220, 260, 300, 340, 380, 420, 460, 500, None],
 'max_features': ['auto', 'sqrt'],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 15.0min finished


{'n_estimators': 1400, 'max_features': 'auto', 'max_depth': 100}


In [94]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

pprint(random_grid)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores

clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
clf_random.fit(X_oversample, y_oversample)
print(clf_random.best_params_)

{'max_depth': [100, 140, 180, 220, 260, 300, 340, 380, 420, 460, 500, None],
 'max_features': ['auto', 'sqrt'],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 17.8min finished


{'n_estimators': 2000, 'max_features': 'auto', 'max_depth': 300}


In [171]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

clf_grid = GridSearchCV(estimator = clf, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1)
clf_grid.fit(X_oversample, y_oversample)
print(clf_grid.best_params_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 12.5min finished


{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 200}


In [172]:
clf5 = RandomForestClassifier(n_estimators= 200, max_features= 3, max_depth= 80,min_samples_leaf=3,min_samples_split=8,random_state = 42)
clf5.fit(X_oversample, y_oversample)
Y_pred = clf5.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

[[434  40]
 [ 34  43]]
0.5180722891566265
0.5584415584415584
              precision    recall  f1-score   support

           0       0.93      0.92      0.92       474
           1       0.52      0.56      0.54        77

    accuracy                           0.87       551
   macro avg       0.72      0.74      0.73       551
weighted avg       0.87      0.87      0.87       551



In [156]:
clf2 = RandomForestClassifier(n_estimators= 2000, max_features= 'auto', max_depth= 300,random_state = 42)
clf2.fit(X_oversample, y_oversample)
Y_pred = clf2.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

[[440  34]
 [ 40  37]]
0.5211267605633803
0.4805194805194805
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       474
           1       0.52      0.48      0.50        77

    accuracy                           0.87       551
   macro avg       0.72      0.70      0.71       551
weighted avg       0.86      0.87      0.86       551



In [157]:
from sklearn.ensemble import RandomForestClassifier

clf1 = RandomForestClassifier(n_estimators= 1400, max_features= 'auto', max_depth= 100,random_state = 42)
clf1.fit(X1_train, y1_train)
Y_pred = clf1.predict(X_test)
print(confusion_matrix(y_test, Y_pred))
print(precision_score(y_test, Y_pred))
print(recall_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

[[453  21]
 [ 47  30]]
0.5882352941176471
0.38961038961038963
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       474
           1       0.59      0.39      0.47        77

    accuracy                           0.88       551
   macro avg       0.75      0.67      0.70       551
weighted avg       0.86      0.88      0.87       551



##This is the Final Output CSV

In [173]:
Y_final = clf5.predict(final_data)
answer = pd.DataFrame(columns = ['PATIENT ID','hospital_outcome'])
answer['PATIENT ID'] = final_id
answer['hospital_outcome'] = Y_final
answer

,PATIENT ID,hospital_outcome
0,1636,0
1,734,1
2,1045,1
3,1728,0
4,1267,0
...,...,...
607,1735,1
608,1807,1
609,1464,0
610,595,0


In [174]:
answer.hospital_outcome.value_counts()

0    542
1     70
Name: hospital_outcome, dtype: int64

In [175]:
answer.to_csv(r'106022113.csv')